In [1]:
import numpy as np
import pickle
import sys
import matplotlib.pyplot as plt
from scipy.optimize import differential_evolution
from pathlib import Path
from bgp_qnm_fits import *

In [13]:
SIMNUMS = [
    "0001",
    "0002",
    "0003",
    "0004",
    "0005",
    "0006",
    "0007",
    "0008",
    "0009",
    "0010",
    "0011",
    "0012",
    "0013",
]
RINGDOWN_START_TIMES = [
    17.0,
    21.0,
    23.0,
    26.0,
    17.0,
    17.0,
    17.0,
    11.0,
    29.0,
    16.0,
    12.0,
    17.0,
    6.0,
]
TRAINING_SPH_MODES = [
    (2, 2),
    (2, 1),
    (3, 3),
    (3, 2),
    (4, 4),
    (2, -2),
    (2, -1),
    (3, -3),
    (3, -2),
    (4, -4),
]

SIM_TRAINING_MODE_RULES = {
    "0001": "PE",
    "0002": "PE",
    "0003": "PE",
    "0004": "PE",
    "0005": "P",
    "0006": "P",
    "0007": "P",
    "0008": "ALL",
    "0009": "E",
    "0010": "P",
    "0011": "P",
    "0012": "P",
    "0013": "ALL",
}

SMOOTHNESS = 16
EPSILON = 1 / 10

# These determine the parameter and training range but do not have to match `analysis times' used later.

TRAINING_START_TIME = -10
TRAINING_END_TIME = 100
TIME_STEP = 0.1

analysis_times = np.arange(
    TRAINING_START_TIME,
    TRAINING_START_TIME + TRAINING_END_TIME,
    TIME_STEP,
)  

# Define training bounds

SIGMA_MAX_LOWER, SIGMA_MAX_UPPER = 0.1, 5
T_S_LOWER, T_S_UPPER = -20, 30
LENGTH_SCALE_LOWER, LENGTH_SCALE_UPPER = 0.1, 5
PERIOD_LOWER, PERIOD_UPPER = 0.1, 5

SMOOTHNESS_LOWER, SMOOTHNESS_UPPER = 0, 30
LENGTH_SCALE_2_LOWER, LENGTH_SCALE_2_UPPER = 0.1, 5
PERIOD_2_LOWER, PERIOD_2_UPPER = 0.1, 5
A_LOWER, A_UPPER = 0, 0.9

BOUNDS_WN = [
    (SIGMA_MAX_LOWER, SIGMA_MAX_UPPER),
]

BOUNDS_GP = [
    (SIGMA_MAX_LOWER, SIGMA_MAX_UPPER),
    (T_S_LOWER, T_S_UPPER),
    (LENGTH_SCALE_LOWER, LENGTH_SCALE_UPPER),
    (PERIOD_LOWER, PERIOD_UPPER),
]

BOUNDS_GPC = [
    (SIGMA_MAX_LOWER, SIGMA_MAX_UPPER),
    (T_S_LOWER, T_S_UPPER),
    (SMOOTHNESS_LOWER, SMOOTHNESS_UPPER),
    (LENGTH_SCALE_LOWER, LENGTH_SCALE_UPPER),
    (PERIOD_LOWER, PERIOD_UPPER),
    (LENGTH_SCALE_2_LOWER, LENGTH_SCALE_2_UPPER),
    (PERIOD_2_LOWER, PERIOD_2_UPPER),
    (A_LOWER, A_UPPER),
]

# Set initial params

# INITIAL_PARAMS_WN = [1.]
# INITIAL_PARAMS_GP = [1.0, 0.0, 1.0, 1.0]
# INITIAL_PARAMS_GPC = [0.5715534011443748, 0.0032311845355438894, SMOOTHNESS, 1.7176362780942858, 0.31558556618927797, 1.7176362780942858, 0.31558556618927797, 0.5]

INITIAL_PARAMS_WN = [0.29127733345656215]
INITIAL_PARAMS_GP = [
    0.2283378440307793,
    18.37394010821784,
    0.8610899535603144,
    0.2605530172829033,
]
# INITIAL_PARAMS_GPC = [0.5678699426741673, 3.3680141572797027, 7.841502124072786, 1.241209026430354, 0.9894982312667636, 0.1064862157208278, 0.139811581920352, 0.5917377132835934]
INITIAL_PARAMS_GPC = [
    0.29443340366568055,
    17.00880319694479,
    8.52342433839235,
    0.992215662729599,
    0.29792754163345136,
    1.4599452640915012,
    3.702622948813973,
    0.8844594560538273,
]

# Define rules for updating params

HYPERPARAM_RULE_DICT_WN = {
    "sigma_max": "multiply",
}

HYPERPARAM_RULE_DICT_GP = {
    "sigma_max": "multiply",
    "t_s": "sum",
    "length_scale": "multiply",
    "period": "multiply",
}

HYPERPARAM_RULE_DICT_GPC = {
    "sigma_max": "multiply",
    "t_s": "sum",
    "sharpness": "replace",
    "length_scale": "multiply",
    "period": "multiply",
    "length_scale_2": "multiply",
    "period_2": "multiply",
    "a": "replace",
}

In [ ]:
with open("param_dict.pkl", "rb") as f:
    param_dict = pickle.load(f)
with open("R_dict.pkl", "rb") as f:
    R_dict = pickle.load(f)


#data_mask = np.ones_like(analysis_times, dtype=bool)
#data_mask[analysis_times > 20] = False

#R_dict_masked = {k:{K:V[data_mask] for K, V in v.items()} for k, v in R_dict.items()}
#analysis_times_masked = analysis_times[data_mask]


In [30]:
sigma_max_range = np.linspace(0.01, 0.6, 20)
liklihoods = np.zeros_like(sigma_max_range)

for i, sigma_max in enumerate(sigma_max_range):

    hyperparam_list = [sigma_max, 23.06664076, 1.09787881, 0.2478821]

    likelihood = get_total_log_evidence(
            hyperparam_list,
            param_dict,
            R_dict,
            HYPERPARAM_RULE_DICT_GP,
            analysis_times,
            kernel_main,
            TRAINING_SPH_MODES,
            SIM_TRAINING_MODE_RULES,
            alpha=2,
            beta=2
        )
    
    liklihoods[i] = likelihood

plt.plot(sigma_max_range, liklihoods)
plt.xlabel("Sigma Max")
plt.ylabel("Likelihood")
plt.show()


KeyboardInterrupt: 